In [129]:
import pickle
import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
# from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
# from tensorflow.keras import Input

from keras import Input

# from keras_contrib.layers import CRF
from tensorflow_addons.layers import CRF
from keras_contrib import losses
from keras_contrib import metrics

NameError: name 'tensorflow' is not defined

In [5]:
df_ar_gk_train = pd.read_csv('ar_gk_train_cleaned.csv')

sents_train = []
temp = []
tags_train = []
temp_tags = []
current_id = 0

for item in df_ar_gk_train.itertuples(index=False):
    if item.sent_id != current_id:
        current_id += 1
        sents_train.append(temp)
        tags_train.append(temp_tags)
        temp = []
        temp_tags = []
    temp.append(item)
    temp_tags.append(item.NER_tags)

In [6]:
df_ar_gk_test = pd.read_csv('ar_gk_test_cleaned.csv')

sents_test = []
temp = []
tags_test = []
temp_tags = []
current_id = 0

for item in df_ar_gk_test.itertuples(index=False):
    if item.sent_id != current_id:
        current_id += 1
        sents_test.append(temp)
        tags_test.append(temp_tags)
        temp = []
        temp_tags = []
    temp.append(item)
    temp_tags.append(item.NER_tags)

In [7]:
all_sents = sents_train + sents_test

In [152]:
all_sents_filtered = []

start = False

for sent in all_sents:
    temp = []
    for token in sent:
        if token.token_text == '-LRB-':
            start = True
            continue
        elif token.token_text == '-RRB-':
            start = False
            continue
        elif not start:
            temp.append(token)
    all_sents_filtered.append(temp)

In [37]:
filtered_train = []
filtered_test = []

start = False

for sent in sents_train:
    temp = []
    for token in sent:
        if token.token_text == '-LRB-':
            start = True
            continue
        elif token.token_text == '-RRB-':
            start = False
            continue
        elif not start:
            temp.append(token)
    filtered_train.append(temp)
            
start = False
            
for sent in sents_test:
    temp = []
    for token in sent:
        if token.token_text == '-LRB-':
            start = True
            continue
        elif token.token_text == '-RRB-':
            start = False
            continue
        elif not start:
            temp.append(token)
    filtered_test.append(temp)

In [41]:
all_filtered[0]

[Pandas(sent_id=0, token_text='1/2', token_lemma='1/2', token_tag='CD', NER_tags='B-QUANTITY'),
 Pandas(sent_id=0, token_text='large', token_lemma='large', token_tag='JJ', NER_tags='SIZE'),
 Pandas(sent_id=0, token_text='sweet', token_lemma='sweet', token_tag='JJ', NER_tags='B-NAME'),
 Pandas(sent_id=0, token_text='red', token_lemma='red', token_tag='JJ', NER_tags='I-NAME'),
 Pandas(sent_id=0, token_text='onion', token_lemma='onion', token_tag='NN', NER_tags='I-NAME'),
 Pandas(sent_id=0, token_text=',', token_lemma=',', token_tag=',', NER_tags='O'),
 Pandas(sent_id=0, token_text='thinly', token_lemma='thinly', token_tag='RB', NER_tags='O'),
 Pandas(sent_id=0, token_text='sliced', token_lemma='sliced', token_tag='VBD', NER_tags='B-STATE')]

In [39]:
all_filtered = filtered_test + filtered_train

In [54]:
all_filtered[5]

[Pandas(sent_id=5, token_text='3/4', token_lemma='3/4', token_tag='CD', NER_tags='B-QUANTITY'),
 Pandas(sent_id=5, token_text='cup', token_lemma='cup', token_tag='JJ', NER_tags='B-UNIT'),
 Pandas(sent_id=5, token_text='white', token_lemma='white', token_tag='JJ', NER_tags='B-NAME'),
 Pandas(sent_id=5, token_text='sugar', token_lemma='sugar', token_tag='NN', NER_tags='I-NAME')]

In [153]:
index_min = np.argmax([len(x) for x in all_sents_filtered])
max_len = len(all_sents_filtered[index_min])

In [154]:
tags = list(set([x.NER_tags for xs in all_sents_filtered for x in xs]))
words = list(set([x.token_text for xs in all_sents_filtered for x in xs]))

In [170]:
tags

['B-STATE',
 'B-NAME',
 'B-QUANTITY',
 'SIZE',
 'I-NAME',
 'I-STATE',
 'I-QUANTITY',
 'I-UNIT',
 'B-UNIT',
 'TEMP',
 'DF',
 'O']

In [155]:
word2index = {word: idx + 2 for idx, word in enumerate(words)}

word2index["UNKNOWN"] = 0

word2index["PADDING"] = 1

index2word = {idx: word for word, idx in word2index.items()}

In [171]:
for k,v in sorted(word2index.items(), key=operator.itemgetter(1))[:10]:
    print(k,v)

UNKNOWN 0
PADDING 1
nan 2
hazelnuts 3
uncooked 4
knead 5
Spinach 6
blanched 7
double-acting 8
hands 9


In [157]:
test_word = "uncooked"

test_word_idx = word2index[test_word]
test_word_lookup = index2word[test_word_idx]

print("The index of the word {} is {}.".format(test_word, test_word_idx))
print("The word with index {} is {}.".format(test_word_idx, test_word_lookup))

The index of the word uncooked is 4.
The word with index 4 is uncooked.


In [172]:
tag2index = {tag: idx + 1 for idx, tag in enumerate(tags)}

tag2index["PADDING"] = 0

index2tag = {idx: word for word, idx in tag2index.items()}

In [175]:
tag2index

{'B-STATE': 1,
 'B-NAME': 2,
 'B-QUANTITY': 3,
 'SIZE': 4,
 'I-NAME': 5,
 'I-STATE': 6,
 'I-QUANTITY': 7,
 'I-UNIT': 8,
 'B-UNIT': 9,
 'TEMP': 10,
 'DF': 11,
 'O': 12,
 'PADDING': 0}

In [188]:
for x in x_train[4]:
    print(index2word[x])

2
jalapeno
peppers
,
seeded
and
minced


In [197]:
[tag2index[tag] for tag in y_train_base[4]]

[3, 2, 5, 12, 1, 12, 6]

In [200]:
x_train_base = [[word.token_text for word in sent] for sent in all_sents_filtered]
y_train_base = [[word.NER_tags for word in sent] for sent in all_sents_filtered]

print("X[0]:", x_train_base[4])
print("y[0]:", y_train_base[4])

x_train = [[word2index[word] for word in sent] for sent in x_train_base]
y_train = [[tag2index[tag] for tag in sent] for sent in y_train_base]

print("X[0]:", x_train[4])
print("y[0]:", y_train[4])

x_train_padded = [sent + [word2index["PADDING"]] * (max_len - len(sent)) for sent in x_train]
y_train_padded = [sent + [tag2index["PADDING"]] * (max_len - len(sent)) for sent in y_train]

print("X[0]:", x_train_padded[4])
print("y[0]:", y_train_padded[4])

TAG_COUNT = len(tag2index)
y_train_one_hot = [ np.eye(TAG_COUNT)[sent] for sent in y_train_padded]

print("X[0]:", x_train_padded[4])
print("y[0]:", y_train_one_hot[4])

X[0]: ['2', 'jalapeno', 'peppers', ',', 'seeded', 'and', 'minced']
y[0]: ['B-QUANTITY', 'B-NAME', 'I-NAME', 'O', 'B-STATE', 'O', 'I-STATE']
X[0]: [1632, 1147, 2125, 1137, 619, 1079, 1641]
y[0]: [3, 2, 5, 12, 1, 12, 6]
X[0]: [1632, 1147, 2125, 1137, 619, 1079, 1641, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y[0]: [3, 2, 5, 12, 1, 12, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
X[0]: [1632, 1147, 2125, 1137, 619, 1079, 1641, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y[0]: [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 

In [219]:
print("X[0]:", x_train_padded[4])
print("y[0]:", y_train_one_hot[4])

X[0]: [1632, 1147, 2125, 1137, 619, 1079, 1641, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y[0]: [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0.

In [202]:
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(x_train_padded, y_train_one_hot, test_size=0.2, random_state=1234)

print("Number of sentences in the training dataset: {}".format(len(X_train_final)))
print("Number of sentences in the test dataset : {}".format(len(X_test_final)))

Number of sentences in the training dataset: 7036
Number of sentences in the test dataset : 1760


In [223]:
y_train_final[1]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 

In [74]:
WORD_COUNT = len(index2word)
DENSE_EMBEDDING = 50
LSTM_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_UNITS = 100
BATCH_SIZE = 256
MAX_EPOCHS = 5

In [206]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow_addons.utils.types import FloatTensorLike, TensorLike
from tensorflow_addons.layers import CRF
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow_addons.optimizers import AdamW

def build_model(max_len = max_len, input_dim = len(word2index),embedding_dim = 300):
  # Model definition
  input = Input(shape=(max_len,))

  # Get embeddings
  embeddings = Embedding(
      input_dim,
      embedding_dim,
      # weights=[embedding_matrix],
    input_length=max_len,
       mask_zero=True,
    trainable=True,
      name = 'embedding_layer'
  )(input)

  # variational biLSTM
  output_sequences = Bidirectional(LSTM(units=50, return_sequences=True))(embeddings)
  output_sequences = Bidirectional(LSTM(units=100, return_sequences=True))(output_sequences)
  # Stacking
  output_sequences = Bidirectional(LSTM(units=50, return_sequences=True))(output_sequences)

  # Adding more non-linearity
  dense_out = TimeDistributed(Dense(25, activation="relu"))(output_sequences)

  # CRF layer
  mask = Input(shape=(max_len,), dtype=tf.bool)
  crf = CRF(13, name='crf')
  predicted_sequence, potentials, sequence_length, crf_kernel = crf(dense_out)

  model = Model(input, potentials)
  model.compile(
      optimizer=AdamW(weight_decay=0.001),
      loss= SigmoidFocalCrossEntropy()) # Sigmoid focal cross entropy loss

  return model

model = build_model()

# Checkpointing
save_model = tf.keras.callbacks.ModelCheckpoint(filepath='ner_crf.h5',
  monitor='val_loss',
  save_weights_only=True,
  save_best_only=True,
  verbose=1
)

# Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=10)

callbacks = [save_model, es]

model.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 25)]              0         
                                                                 
 embedding_layer (Embedding  (None, 25, 300)           678300    
 )                                                               
                                                                 
 bidirectional_27 (Bidirect  (None, 25, 100)           140400    
 ional)                                                          
                                                                 
 bidirectional_28 (Bidirect  (None, 25, 200)           160800    
 ional)                                                          
                                                                 
 bidirectional_29 (Bidirect  (None, 25, 100)           100400    
 ional)                                                   

In [204]:
X_train_final = np.array(X_train_final)
X_test_final = np.array(X_test_final)
y_train_final = np.array(y_train_final)
y_test_final = np.array(y_test_final)

In [138]:
print(X_train[0].shape)
print(y_train[0].shape)

(25,)
(25, 13)


In [207]:
history = model.fit(
  X_train_final,
  y_train_final,
  batch_size=64,
  epochs=20,
  validation_data=(X_test_final, y_test_final),
  callbacks=callbacks,
  shuffle=True
)

Epoch 1/20


110/110 [==============================] - ETA: 0s - loss: 0.1862
Epoch 1: val_loss improved from inf to 0.06156, saving model to ner_crf.h5
110/110 [==============================] - 114s 739ms/step - loss: 0.1862 - val_loss: 0.0616
Epoch 2/20
110/110 [==============================] - ETA: 0s - loss: 0.0545
Epoch 2: val_loss improved from 0.06156 to 0.04671, saving model to ner_crf.h5
110/110 [==============================] - 63s 570ms/step - loss: 0.0545 - val_loss: 0.0467
Epoch 3/20
110/110 [==============================] - ETA: 0s - loss: 0.1254
Epoch 3: val_loss did not improve from 0.04671
110/110 [==============================] - 66s 596ms/step - loss: 0.1254 - val_loss: 0.0792
Epoch 4/20
110/110 [==============================] - ETA: 0s - loss: 0.0667
Epoch 4: val_loss did not improve from 0.04671
110/110 [==============================] - 62s 566ms/step - loss: 0.0667 - val_loss: 0.0590
Epoch 5/20
110/110 [==============================] - ETA: 0s - loss: 0.0559
Epoch 5: 

In [212]:
predictions = model.predict(X_train_final)

220/220 [==============================] - 30s 137ms/step


In [213]:
predicted_tags = np.argmax(predictions, axis=-1)
predicted_tags = [[index2tag[tag] for tag in sample] for sample in predicted_tags]

flatten_predicted = [tag for sample in predicted_tags for tag in sample]
flatten_true = [index2tag[np.argmax(tag)] for sample in y_train_final for tag in sample]

unique_tags = list(set(flatten_true + flatten_predicted))

In [235]:
from collections import Counter

Counter(flatten_predicted)

Counter({'PADDING': 141134,
         'B-NAME': 8983,
         'O': 8590,
         'B-QUANTITY': 7036,
         'B-UNIT': 6774,
         'I-NAME': 3382,
         'I-QUANTITY': 1})

In [233]:


from sklearn.metrics import classification_report
print("Classification Report:\n ", classification_report(flatten_predicted, flatten_true, labels=list(set(flatten_true))))



c:\Users\Admin\miniconda4\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\miniconda4\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
                precision    recall  f1-score   support

     B-STATE       0.00      0.00      0.00         0
      B-NAME       0.71      0.54      0.61      8983
  B-QUANTITY       0.99      0.94      0.97      7036
        SIZE       0.00      0.00      0.00         0
      I-NAME       0.45      0.63      0.52      3382
     I-STATE       0.00      0.00      0.00         0
  I-QUANTITY       0.02      1.00      0.04         1
     PADDING       1.00      1.00      1.00    141134
           O       0.75      0.58      0.65      8590
      I-UNIT       0.00      0.00      0.00         0
        TEMP       0.00      0.00      0.00         0
          DF       0.00      0.00      0.00         0
      B-UNIT       0.95      0.76      0.85      6774

    accuracy                           0.94    175900
   macro avg       0.37      0.42      0.36    175900
weighted avg       0.96      0.94      0.95    175900



c:\Users\Admin\miniconda4\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [145]:
predicted_tags[0]

['B-QUANTITY',
 'B-UNIT',
 'B-NAME',
 'O',
 'B-STATE',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING',
 'PADDING']

In [146]:
y_test[0]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 